# Reading 2: Testing

Objectives

* understand the value of testing, and know the process of **test-first programming** ;
* be able to judge a test suite for correctness, thoroughness, and size;
* be able to design a test suite for a function by **partitioning** its input space and choosing good test cases;
* be able to judge a test suite by measuring its **code coverage**; 
* understand and know when to use **black box** vs. **glass box** testing, **unit tests** vs. integration tests, and **automated regression testing**.

## Validation
测试（test）更加通用的叫法是验证（validation）

## 为什么软件测试很难
常见的测试策略对于软件来说很难奏效

* 详尽的测试
  * 测试32为浮点数乘法：a*b,需要准备$ 2^{32} * 2^{32}$个测试用例
* 随意测试
* 尝试一下，看看它是否有效”）不太可能发现错误，除非程序有太多错误，以至于任意选择的输入更有可能失败而不是成功。它也不会增加我们对程序正确性的信心。
* 随机或者基于统计测试（Random or statistical testing ）
  * 一般工业制品使用的测试方法，比如随机取样，测试生产的驱动器的1%的成品。
  * 在24小时内开关冰箱门10000次，来模拟10年冰箱冰箱们的开关情况。
  * 软件的区别
    * 通常在多数的连续情况下运行良好，而在某一个单点（例如某一些特殊值：0,max,min）
    * 堆栈溢出、内存不足错误和数字溢出错误往往会突然发生，并且总是以相同的方式发生，而不是出现概率变化。
    * 物理系统中，经常有明显的证据表明系统正在接近故障点（桥梁中的裂缝），或者故障概率分布在故障点附近（因此，统计测试甚至可以在故障点出现之前观察到一些故障）。


### Needle in a haystack
```ts
/** 
 * @param bits an array of 32 true/false values
 * @returns the Boolean AND of all values in the array
 */
function andAll(bits: Array<boolean>): boolean {
    let result = bits[0];
    for (let i = 1; i < 31; i++) {
        result = result && bits[i];
    }
    return result;
}

```
这段代码中在 i< 31存在一个错误,应该是 **<= 1**。

如果想通过使用随机测试来检测出这个bug,需要一个特殊的测试用例: bits[31]=false, bits[0]...bits[30]全部为true，概率是$\displaystyle \frac{1}{2^{32}}$。

**随机测试被用作软件工程中的一种工具（特别是为了增加对常见情况行为足够可靠或有效的信心，而不仅仅是为了发现错误），但最好将其与系统测试结合使用。**，所以不要只是为了发现bug使用随机测试。

## Test-first programming

术语：
* 模块（module）是软件系统的一部分，可以与系统的其余部分分开设计、实现、测试和推理。在本文中，我们将重点关注作为函数的模块。在以后的阅读中，我们将拓宽视野，思考更大的模块，例如具有多种交互方法的类。

* 规格（或规范，specification）描述模块的行为。对于函数，规范给出了参数的类型以及对它们的任何附加约束（例如sqrt的参数必须是非负的）。它还给出了返回值的类型以及返回值与输入的关系。在 TypeScript 代码中，**规范由函数签名和其上方描述其功能的注释组成**。

* 一个模块有提供行为的实现，客户端使用该模块。对于函数来说，**实现**是函数的主体，而客户端是调用该函数的其他代码（每一个测试用例就是一个客户端）。模块的规范限制了客户端和实现。

* **测试用例**是输入的特定选择，以及规范所需的预期输出行为。

* __测试套件__（test suit） 是模块的一组测试用例（多个测试用例）。

测试优先的编码，在编写任何代码之前就编写了规范和测试。单个功能的开发按以下顺序进行：

1. Spec：编写函数的规范。
2. 测试：编写执行规范的测试。
3. 实现：编写实现。

一旦测试通过，说明编码任务完成（不是写完了代码就算完成了:smile:👍）

测试优先编程的最大好处是远离bug。不要等到开发结束时才进行测试，此时您有大量未经验证的代码。**将测试留到最后只会使调试时间更长、更痛苦，因为错误可能存在于代码中的任何地方** 😢😢😢。在开发代码时测试代码要愉快得多。


## Systematic testing
系统性测试使用一定的原则来挑选测试用例，希望测试用例具有以下的属性：
1. 正确（correct）
   * 正确的测试套件是规范的合法客户端，它毫无怨言地接受规范的所有合法实现。可以自由地更改模块的内部实现方式，而不必更改测试套件
2. 彻底（thorough）
   * thorough指的是错误的种类，并不只是单纯的测试用例的数量。
   * 他主要是让错误的实现通过测试用例暴露出来，即在有缺陷的实现上失败。
3. 小（small）
      * 一个小的测试套件，只有很少的测试用例，编写起来会更快，而且如果规范发生变化，也更容易更新。小型测试套件的运行速度也更快。如果测试套件小而快，您将能够更频繁地运行测试。

使用测试优先的策略还有一个好处，就是一开始就让你带上帽子（莫名的。梗？）。因为人往往对自己费力写的代码充满“呵护”，用一些脆弱的测试让代码正常运行。

## Choosing test cases by partitioning
定义：

将函数的输入空间（所有可能的合法的输入）划分为不同（彼此不相交的集合）的子域（它属于函数的定义域，是一组集合）。partition就是所有subdomain的集合，覆盖了输入空间。

![](ref/lect2/2023-07-11-15-21-41.png)

子域背后的想法是将输入空间划分为相似输入的集合，程序在这些输入上具有相似的行为，这样我们只需要测试每个集合的一个代表。这种方法通过选择不同的测试用例，并强制测试探索偶然或随机测试可能无法到达的输入空间区域，充分利用了有限的测试资源。

出于分区的目的，程序的输入空间仅包括合法输入，为其指定正确的行为（因此可测试）。程序没有定义行为的输入不是合法输入空间的一部分，不应包含在分区中。

### 示例
#### abs()
```js
/**
 * ...
 * @param a  the argument whose absolute value is to be determined
 * @returns the absolute value of the argument.
 */
function abs(a: number): number
```
从数学上讲，这是以下类型的函数：

绝对值：数字→数字
![](ref/lect2/2023-07-11-15-29-16.png)

该函数具有一维输入空间，由a的所有可能值组成。**考虑绝对值函数的行为方式**，我们可以首先将输入空间划分为这两个子域：{a ≥ 0 } 和 { a < 0}。在第一个子域上，abs应返回原值。在第二个子域上，abs应该取相反数。

一般描述集合使用{a | ...},此处直接写作 **// partition: a >= 0; a < 0**

为了为我们的测试套件选择测试用例，我们从分区的每个子域中选择一个任意值a，例如：

* a = 17 覆盖子域a ≥ 0
* a = -3 覆盖子域a < 0

#### max()
```js
/**
 * ...(省略了行为描述)
 * @param a  an argument
 * @param b  another argument
 * @returns the larger of a and b.
 */
function max(a: number, b: number): number
```
从数学上讲，这是两个参数的函数：

最大值：数量×数量→数量
![](ref/lect2/2023-07-11-15-39-57.png)

这是一个二维输入空间，由所有整数对 (a , b) 组成。对其进行分区。从规范来看，选择子域 { (a,b) | a < b} and {(a,b) | a > b}是有意义的  ，因为规范对每个子域要求不同的行为：前一个子域中的输入应返回b，而后一个子域中的输入应返回a。但不能就此止步，因为这些子域还不是输入空间的partition。分区必须完全覆盖可能的输入集。所以我们需要添加 { (a,b) | a = b }，其中函数可以返回a或b。简洁地表达，分区如下所示：

// partition: a < b; a > b; a = b

测试套件可能是：

* ( a , b ) = (1, 2) 覆盖a < b
* ( a , b ) = (10, -8) 覆盖a > b
* ( a , b ) = (9, 9) 覆盖a = b


总而言之，要形成分区，子域应具有三个理想的属性：

* 子域应该是不相交的，它们之间没有交集；
* 子域应该是完整的，以便它们的并集覆盖合法的输入空间；
* 子域应该是非空的，以便可以从每个子域中选择一个合法的测试用例。

### 在分区中包含边界(boundaries)
错误经常发生在子域的边界之间，例如：

* 0是正数和负数的分界线
* 数字类型的最大值和最小值，例如Number.MAX_SAFE_INTEGERorNumber.MAX_VALUE
* 集合类型为空，例如空字符串、空数组或空集
* 序列的第一个和最后一个元素，例如字符串或数组

为什么边界处经常出现bug？
* 一个原因是程序员经常犯差一的错误，
  * 例如编写<=而不是<，或者将计数器初始化为 0 而不是 1。
  * 某些边界可能需要在代码中作为特殊情况进行处理。
* 另一个问题是边界可能是代码行为中不连续的地方。
  * 例如，当number用作整数的变量超的Number.MAX_SAFE_INTEGER时，它会突然开始失去精度。

将边界合并到分区中的的单元素子域，以便测试套件包含边界值作为测试用例。对于abs，我们将为相关边界添加一个子域：

a = 0，因为abs对正数和负数的表现不同
然后，我们原来的两个子域将缩小以排除边界值：{ a | a > 0 } 和 { a | 一个<0}。

这仍然是abs的输入空间的partition：三个子域是不相交的并且完全覆盖输入空间。一种紧凑的编写方法如下所示：
```js
// partition:
//   a < 0
//   a = 0
//   a > 0
```
Our test suite might then be:

* a = 0
* a = 17 to cover the subdomain a > 0
* a = -3 to cover the subdomain a < 0

### BigInt乘法
BigInt是TypeScript内置的一个类型，可以表示任意大小的整数，不像number类型只有有限的范围。BigInt支持常规的整数运算，如+、-、*、/和**。

乘法：BigInt × BigInt → BigInt

我们再次有一个二维输入空间，由所有整数对 ( a , b ) 组成。考虑符号规则如何与乘法配合使用，我们可以从这些子域开始：

* a和b均为正数
* a和b均为负数
* a为正，b为负
* a为负，b为正
  
我们还应该检查一些乘法的边界值：

* a或b为 0，因为结果始终为 0
* a或b为 1，乘法的单位值

最后，因为BigInt的目的是表示任意大的整数，所以我们应该确保尝试非常大的整数——至少大于Number.MAX_SAFE_INTEGER，即 $2^{53} -1$，一个 16 位十进制整数。

* a或者b在数量级要“大”或者“小”（小，意味着number足够表示的值；大，意味着超过number表示的范围）

将所有这些观察结果集中到整个 ( a , b ) 空间的单个分区中。我们将独立地选择a和b：
1. 0
2. 1
3. 小正整数（≤Number.MAX_SAFE_INTEGER且 > 1）
4. 小负整数（≥Number.MIN_SAFE_INTEGER且 < 0）
5. 大正整数 (> Number.MAX_SAFE_INTEGER)
6. 大负整数 (< Number.MIN_SAFE_INTEGER)

因此，这将产生 6 × 6 = 36 个子域，用于划分整数对的空间。

![](ref/lect2/2023-07-11-19-33-11.png)

从上式分出来的测试套件（每一个点，代表着一个测试用例）：
* ( a , b ) = (0, 0) 覆盖 (0, 0)
* ( a , b ) = (0, 1) 覆盖 (0, 1)
* ( a , b ) = (0, 8392) 覆盖 (0, 小正整数)
* ( a , b ) = (0, -7) 覆盖 (0, 小负整数)
* ……
* ( a , b ) = $(-10^{60} , -10^{810})$ 覆盖（大负值，大负值）

### 使用多分区（partitions）

到目前为止，示例仅在整个输入空间中使用了一个分区（一组不相交的子域）。对于具有多个参数的函数，这可能会变得昂贵。每个参数可能有有​​趣的行为变化和几个边界值，因此根据每个参数的行为的笛卡尔积形成输入空间的单个分区会导致结果测试套件大小的组合爆炸。通过BigInt乘法示例可以看到了这一点，使用笛卡尔积划分有 6 × 6 = 36 个子域，需要 36 个测试用例来覆盖。对于具有n 个参数的函数，笛卡尔积方法会生成一个大小为n指数的测试套件，这对于手动测试创作很快就变得不可行。

另一种方法是将每个输入a和b的特征视为输入空间的两个单独的分区。
一个分区仅考虑a的值：

( a , b ) 使得a = 0, 1，小正值，小负值，大正值，大负值。

另一个分区只考虑b的值：

( a , b ) 使得b = 0, 1，小正值，小负值，大正值，大负值。
![](ref/lect2/2023-07-11-19-45-10.png)

如图所示，图上的样本用例一共是6个，分别看a，b，满足了二者所有的子域。

a,b单独的分区可以紧凑的写成如下形式：

```js
// partition on a:
//   a = 0
//   a = 1
//   a is small integer > 1
//   a is small integer < 0
//   a is large positive integer
//   a is large negative integer
//      (where "small" fits in a TypeScript number, and "large" doesn't)
// partition on b:
//   b = 0
//   b = 1
//   b is small integer > 1
//   b is small integer < 0
//   b is large positive integer
//   b is large negative integer
```
以上每一个子域都能被一个测试用例所覆盖，同时现在一个单一的测试用例能够覆盖来自不同分区的多个subdomain，比如图上的一个点，即代表着a的一个domain，也代表b的一个domian。这提高了测试用例的效率。

将a和b独立分开可能会导致无法测试它们之间的相互作用。例如，乘法中的符号处理是一个可能的错误源，结果的符号取决于a和b的符号,但是我们可以添加一个额外的分区来捕捉这种交互：
![](/ref/lect2/2023-07-11-21-53-29.png)
```js
// partition on signs of a and b:
//   a and b are both positive
//   a and b are both negative
//   a positive and b negative
//   a negative and b positive
//   one or both are 0
```
现在我们有三个分区，每个分区有 6、6 和 5 个子域，但我们不需要 6 × 6 × 5 测试用例的笛卡尔积来覆盖它们。包含 6 个精心挑选的测试用例的测试套件可以覆盖所有三个分区的子域。
在仔细选择测试用例的情况下，额外的分区应该只需要很少（如果有的话）的额外测试用例。

有时，我们可能希望在多个分区上使用笛卡尔乘积方法，以产生一个更全面的测试套件。但即使在这种情况下，笛卡尔积也可能比我们预期的要小。当来自不同分区的子域相互排斥时，笛卡尔乘积将不包括该特定子域组合的子域。我们将在下面的练习中看到一个例子。

然而，作为测试优先编程的起点，一个小的测试套件涵盖了几个经过深思熟虑选择的分区的每个子域，在规模和彻底性之间取得了很好的平衡。测试套件可以通过玻璃箱（glass box）测试、代码覆盖率测量（code coverage measurement）和回归测试（egression testing）进一步扩展。


### reading exercise

#### One partition vs. multiple partitions
```js
// partition on a:
//   a = 0
//   a = 1
//   a is small integer > 1
//   a is small integer < 0
//   a is large positive integer
//   a is large negative integer
//      (where "small" fits in a TypeScript number, and "large" doesn't)
```
该划分实际上结合了几个不同的问题： a的符号、 a的大小（小或大）以及边界值 0 和 1。

相反，我们可以将这些问题视为独立的分区。从以下选项中，选择一个作为合法分区的子集，并且它们一起将捕获相同的问题：
- [ ] partition on a: 0, 1
- [ ] partition on a: 0
- [ ] partition on a: 1
- [x] partition on a: 0, positive, negative
- [ ] partition on a: positive, negative
- [x] partition on a: 1, !=1
- [x] partition on a: small (fits in a number), large (doesn't fit in a number)

* **注意，每一个分区（partition）必须覆盖在整个输入空间**
* 围绕边界值进行分区尤为重要，例如1, !=1分区。该分区确保我们不仅在特殊值（a = 1）处进行测试，而且还在远离边界（a != 1）的其他地方进行测试。

#### 覆盖每个子域与覆盖笛卡尔积
```js
// partition on a:
//   a = 0
//   a = 1
//   a is small integer > 1
//   a is small integer < 0
//   a is large positive integer
//   a is large negative integer
//      (where "small" fits in a number, and "large" doesn't)
```
该分区有6个子域，可以使用6个测试用例来覆盖分区。

但如果采用单独分区
```js
// partition on a: 0, positive, negative
// partition on a: 1, !=1
// partition on a: small (fits in a number), large (doesn't fit in a number)
```
此处将a划分为单独的三个分区
如果想要覆盖三个分区的所有子域，a需要多少不同的值？
solution：3（找子域最多的）

如果我们想要覆盖这三个分区的笛卡尔积，a我们需要多少个不同的值？

solution：可以采用画tree的方式
![](ref/lect2/2023-07-11-22-42-55.png)

有些子域之间是互斥的（即无法同时存在，所有比一般的笛卡尔积的数量少）。
在多个分区上使用笛卡尔积方法，以生成更全面的测试套件

#### Partitions expressed on outputs
有时根据函数的输出来考虑和编写输入空间的partition是很方便的，因为行为的变化在那里可能更明显。

```js
// partition on a*b: 0, positive, negative
```
简写为： { (a,b) | a*b = 0 }，  { (a,b) | a*b > 0 }  { (a,b) | a*b < 0 }

使用这种方法，需要多少测试用例来覆盖以下三个分区？

// partition on a: 0, positive, negative
// partition on b: 0, positive, negative
// partition on a*b: 0, positive, negative

solution: 4

三个测试用例还不够，还需要一个包含 第4个测试套件的示例，其中包括：

(0, 0) 覆盖a = 0, b = 0,a*b = 0
(3, 5) 覆盖a positive, b positive,a*b positive
(-9, -13) 覆盖a negative, b negative（和a*b positive，即使我们已经覆盖它）
(-39, 7) 覆盖a*b negative（并且a negative和b positive，即使我们已经覆盖了它）

需要多少块才能覆盖以下两个分区？

// partition on value of abs(x): same as x, different from x
// partition on sign of abs(x): 0, positive
solution：2

这里，两个测试用例就足够了：

x=0 覆盖abs(x) same as x, 和abs(x) = 0
x=-15 覆盖abs(x) different from x, 和abs(x) positive
…尽管我们可能对这个测试套件感到不舒服，因为它实际上从未尝试过任何正整数x。在标志上加上一个分区，x会更彻底。

关于此练习的警告。尽管这些分区是用函数的输出（分别为a*b和abs(x)）来表示的，但它们仍然对输入空间进行分区，即分别为参数a和b或x。将函数的输出视为空间的附加维度，或者使用函数体内定义的局部变量作为空间的维度是不正确的（即分区只能对输入空间，不能添加其他的空间）。

#### partition
```js
/**
 * Reverses the end of a string.
 *
 *                          012345                     012345
 * For example: reverseEnd("Hello, world", 5) returns "Hellodlrow ,"
 *                               <----->                    <----->
 *
 * With start === 0, reverses the entire text.
 * With start === text.length, reverses nothing.
 *
 * @param text    string that will have its end reversed
 * @param start   the index at which the remainder of the input is reversed.
 *                Requires start to be an integer, 0 <= start <= text.length
 * @returns input text with the substring from `start` to the end of the string reversed
 */
function reverseEnd(text: string, start: number): string
```
以下哪项是start参数的合理划分？

- [ ] start = 0; start = 5; start = 100
- [ ] start < 0; start = 0; start > 0
- [x] start = 0; 0 < start < text.length; start = text.length
- [ ] start < text.length; start = text.length; start > text.length

0、5、100 不是分区。分区应该是可能的起始值的整个空间的划分，而不是特定的测试用例。

start < 0和start > text.length不是该函数的合法输入，因此将它们用作分区中的子域是不合理的。测试用例必须遵守功能规范的要求。

以下哪项是text参数的合理划分？

- [ ] text contains some letters; text contains no letters, but some numbers; text contains neither letters nor numbers
- [x] text.length = 0; text.length > 0
- [x] text.length = 0; text.length-start is odd; text.length-start is even (> 0)
- [ ] text is every possible string from length 0 to 100
- [ ] text is "" or "Hello, world"
- [ ] text should be the empty string in some test case

字母和数字对此函数的行为并不重要，因此对该属性进行分区没有用。

然而，长度是一个有用的分区，因为它可以与start参数交互。

按偶数和奇数长度进行分区也是合理的，因为反转奇数长度的子字符串与偶数长度的字符串（所有元素都必须移动）具有不同的行为（因为它将中间元素保留在原位）。

划分为所有可能长度 0 到 100 的字符串将产生太多的测试用例。它也不是分区，因为它不包含长度超过 100 个字符的字符串。

"Hello, world"和""是我们可以选择的特殊点text。它们不形成输入的分区text。

同样，尽管包含空字符串作为 的边界值很重要text，但它本身并不是一个分区。



## 自动化单元测试

什么是test driver：对你的module运行测试代码的代码（也叫做test harness or test runner）
测试驱动程序不应该是一个交互式程序，提示输入并打印出结果让你手动检查。
相反，测试驱动应该在固定的测试用例中调用模块本身，并自动检查结果是否正确。
测试驱动的结果应该是 "所有测试OK "或者 "这些测试失败：......"。。

大多数编程语言都至少有一个流行的库用于编写自动化单元测试，包括用于 Java 的JUnit和用于 Python 的unittest 。对于 JavaScript 和 TypeScript，流行的选择是Mocha。

一个Mocha单元测试被写成对函数it()的调用，测试名是它的第一个参数，测试体（写成函数表达式）是第二个参数。测试体通常包含一个或多个对被测模块的调用，
然后使用assert，assert.strictEqual或assert，throws等断言函数检查结果。
例如，max的单个单元测试可能是这样的

```ts
it("covers a < b", function() {
  assert.strictEqual(Math.max(1, 2), 2);
});
```

请注意，assert.strictEqual 的参数顺序很重要。第一个参数应该是实际结果——代码实际做了什么。第二个参数应该是预期结果——通常是一个常量，测试希望看到的结果。
如果你把它们调换一下，那么当测试失败时，Mocha可能会产生一个令人困惑的错误信息。
断言也可以接受一个可选的消息字符串作为最后一个参数，你可以用它来使测试失败更清晰。

要将一组单元测试集合成一个测试套件，将它们放在调用describe()中。例如，上面我们为max选择的测试在Mocha中实现时可能是这样的：

```ts
describe("Math.max", function() {
  it("covers a < b", function() {
    assert.strictEqual(Math.max(1, 2), 2);
  });

  it("covers a = b", function() {
    assert.strictEqual(Math.max(9, 9), 9);

  it("covers a > b", function() {
    assert.strictEqual(Math.max(10, -9), 10);
  });
});
```
测试套件可以包含任意数量的函数，当您使用 Mocha 运行测试套件时，这些函数会独立it()运行。如果测试用例中的断言失败，则该测试用例立即返回，并且 Mocha 会记录该测试的失败。即使一项测试失败，套件中的其他测试仍将运行。

请注意，像 Mocha 这样的自动化测试框架可以轻松运行测试，但您仍然必须自己想出好的测试用例。 自动测试生成是一个难题，仍然是计算机科学研究的一个活跃主题。

### Asserting true

```ts
it("should return a number found in the set", function () {
  const set = new Set<number>([293, 384, 10, 5, -3, 99]);
  const result: number = pickRandomly(set);
  /* ASSERTION HERE */
});

```
哪个断言放在代码所示的位置/* ASSERTION HERE */既正确又对调试有用？

- [ ] assert.strictEqual(result, -3);
- [ ] assert(set.has(result));
- [x] assert(set.has(result), "expected result from {" + Array.from(set) + "} but actually was " + result);

assert()测试其参数是否为 true 表达式，但只能访问布尔值本身，而不能访问导致问题的原始集合，因此断言失败时的输出如下所示：

AssertionError: set.has(result)

这对于调试来说不是很有帮助。

每个断言函数都接受一个可选的消息参数，如第二次调用所示assert。当所选择的断言失败时，它可能会这样说：

AssertionError: expected result from {293,384,10,5,-3,99} but actually was 0

--- 

对于字符串和数字等内置类型，strictEqual 所使用的比较方法正如我们所期望的那样有效，但对于数据结构，它却无能为力。例如，下面所有的断言都失败了：

* strictEqual([], [])
* strictEqual([1], [1])
* strictEqual(new Set([1]), new Set([1]))
正如我们在后面的阅读中将看到的，它们失败的原因是充分的：这些可变数组和集合是否 "相等 "是一个相当棘手的问题。因此，当我们测试一个返回这种结构的函数时，我们有两种选择：

使用assert.deepStrictEqual，它执行相等比较，认为具有相同内容的数组、集合、映射和对象字面都是相等的。当我们确切知道正确的结构是什么时，这是最有用的。但是assert.deepStrictEqual有一个危险的注意事项：只能用它来比较内置数组、集合、映射和对象字面的结构，而不能比较其他任何结构。

在结构的属性上使用简单的assert和assert.strictEqual断言。在前面的练习中，当有多个正确输出时，这是最有用的。

### Asserting deep
如果我们想断言 value actual是Set带有单个元素的 "hello"，我们可以使用：

assert.deepStrictEqual(actual, new Set(["hello"]));

…或者我们可以使用：（选择一组必要且充分的正确断言）

- [x] assert(actual.has("hello"));
- [ ] assert( ! actual.has("goodbye"));
- [x] assert.strictEqual(actual.size, 1);
- [ ] assert.strictEqual(actual.values(), ['hello']);



## Documenting your testing strategy

最好写下你用来创建测试套件的测试策略：分区、它们的子域，以及每个测试用例选择覆盖哪些子域。写下测试策略可以让读者更清楚地了解测试套件的全面性。

在describe()函数顶部的注释中记录分区和子域。例如，要记录我们测试max的策略，我们可以这样写

```ts
describe("max", function() {
  /*
   * Testing strategy
   *
   * partition:
   *   a < b
   *   a > b
   *   a = b
   */

   it(...); // test cases here
   it(...);
   it(...);
   ...
});
```

每个测试用例应按其涵盖的子域命名，例如：

```ts
it("covers a < b", function() {
  assert.strictEqual(Math.max(1, 2), 2);
});
```

大多数测试套件将有多个分区，并且大多数测试用例将覆盖多个子域。例如，以下是BigInt使用七个分区的乘法策略：
```ts
describe("multiplication", function() {
  /*
   * Testing strategy
   *
   * cover the cartesian product of these partitions:
   *   partition on a: positive, negative, 0
   *   partition on b: positive, negative, 0
   *   partition on a: 1, !=1
   *   partition on b: 1, !=1
   *   partition on a: small (fits in a TypeScript number), or large (doesn't fit)
   *   partition on b: small, large
   *
   * cover the subdomains of these partitions:
   *   partition on signs of a and b:
   *      both positive
   *      both negative
   *      different signs
   *      one or both are 0
   */

  it("covers a = 1, b != 1, a and b have same sign", function() {
    assert.strictEqual(BigInt(1) * BigInt(33), BigInt(33));
  });

  it("covers a is positive, b is negative, "
      + "a fits in a number, b fits in a number, "
      + "a and b have different signs", function() {
    assert.strictEqual(BigInt(73) * BigInt(-2), BigInt(-146));
  });

  ...
});
```

## Black box and glass box testing
black box意味着只从规范中选择测试用例，而不是函数的实现。这就是我们迄今为止在示例中一直在做的。我们在abs、max和BigInt乘法中划分并寻找边界，而没有查看这些函数的实际代码。事实上，如果我们遵循测试优先的编程方法，我们甚至还没有编写这些函数的代码。

glass box 测试意味着在选择测试用例时要了解函数是如何实际实现的。例如，如果实现根据输入选择不同的算法，那么您应该围绕选择不同算法的点进行分区。如果实现保留了一个内部缓存，可以记住之前输入的答案，那么您就应该测试重复输入以查看缓存是否正常工作。

对于 BigInt 乘法，当我们最终实现它时，我们可能决定将整数表示为十进制数组。这一决定为函数的行为引入了新的边界。两个个位数相乘是微不足道的，但两位数或更多位数的数组需要更复杂的多位数相乘。因此，对该实现的玻璃箱测试可能会根据内部数组的长度引入新的分区。（对于BigInt来说，更有效的表示方法可能是32位无符号整数数组，如Uint32Array，但原理是一样的——基数为$2^{32}$的数字序列，而不是基数为10的数字序列）。

在进行glass box测试时，您必须注意您的测试用例不要求规范中没有明确要求的特定实现行为。例如，如果规范说 "如果输入格式不正确，则抛出异常"，那么您的测试就不应该仅仅因为当前的实现是这样做的，就特别检查TypeError异常。在这种情况下，规范允许抛出任何异常，所以你的测试用例同样应该是通用的，这样才是正确的，并且保留了实现者的自由。我们将在 "规范 "课程中详细介绍这一点。

```ts
/**
 * Sort an array of numbers in nondecreasing order.  Modifies the array so that
 * values[i] <= values[i+1] for all 0 <= i < values.length-1
 */
function sort(values: Array<number>): void {
    // choose a good algorithm for the size of the array
    if (values.length < 10) {
        radixSort(values);
    } else if (values.length < 1_000_000_000) {
        quickSort(values);
    } else {
        mergeSort(values);
    }
}
```
以下哪些测试用例可能是玻璃盒测试产生的边界值？

- [ ] values = []（空数组）
- [ ] values = [1, 2, 3]
- [x] values = [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
- [ ] values = [0, 0, 1, 0, 0, 0, 0]
当数组的长度为 10 时，该函数从一种排序算法（基数排序）切换到另一种排序算法（快速排序）。这使得长度为 10 的数组成为函数行为的边界值。但这个边界从函数的规范（黑匣子）中不可见，只能从其实现（玻璃匣）中看出。

另一方面，每当您使用数组类型时，空数组都是一个边界值，因此不需要galss box测试来发现它。

根据此处任何可见证据，无论是在规范还是实现代码中，其他数组都不是边界值



## Coverage

评价测试套件的一种方法是询问它对程序进行了多彻底的测试。这个概念叫做覆盖率。下面是三种常见的覆盖率：

语句覆盖率：每个语句是否至少被一个测试用例运行？

分支覆盖率：对于程序中的每一个控制分支（例如if or while或?:三元表达式），分支的每一侧是否至少有一个测试用例？

路径覆盖率：每个可能的分支组合--程序中的每条路径--是否都被至少一个测试用例覆盖？

Istanbul代码覆盖工具的输出示例:
![](ref/lect2/2023-07-12-14-17-07.png)
分支覆盖率比语句覆盖率强（即需要更多的测试来实现），路径覆盖率比分支覆盖率强。在工业领域，100%的语句覆盖率是一个常见的目标，但由于防御代码（如 "不应该到达这里 "的断言）无法实现，即使是100%的语句覆盖率也很少能实现。100%的分支覆盖率是非常理想的，安全关键型行业代码的标准甚至更加苛刻（例如，MC/DC、修改条件/决策覆盖率）。不幸的是，100%的路径覆盖是不可行的，需要指数大小的测试套件来实现。

标准的测试方法是增加测试，直到测试套件达到足够的语句覆盖率：即程序中的每一条可达到的语句都至少被一个测试用例执行。在实践中，语句覆盖率通常由代码覆盖率工具来衡量，该工具会统计测试套件运行每条语句的次数。有了这样的工具，玻璃箱测试就很容易了；您只需测量黑盒测试的覆盖率，然后添加更多的测试用例，直到所有重要的语句都被执行为止。

一个很好的JavaScript和TypeScript代码覆盖工具是Istanbul/nyc，如右图所示。在该工具的HTML输出中，被测试套件执行过的行在左侧空白处有一个绿色标记（也显示了该行被执行的次数）。尚未覆盖的行用红色标出。如果一行包含一个分支，而该分支只在一个方向上被执行过--始终为真但从未为假，或者反之--则会有一个I或E图标，表示哪条路径从未被执行过（I表示 "if"，E表示 "else"）。如果您从覆盖工具中看到了右边的结果，那么您的下一步就是创建一个测试用例，使if测试至少一次为真，并将其添加到测试套件中，这样红线就变成了绿线。

## Unit and integration testing

到目前为止，我们一直在讨论隔离测试单个模块的单元测试。隔离测试模块可以使调试更加容易。当一个模块的单元测试失败时，你可以更确信错误是在该模块中发现的，而不是在程序的任何地方。

与单元测试相反，集成测试测试模块的组合，甚至整个程序。如果你只有集成测试，那么当测试失败时，你必须寻找错误。它可能在程序的任何地方。集成测试仍然很重要，因为程序可能在模块之间的连接处失败。例如，一个模块期望的输入可能与实际从另一个模块得到的不同。但是如果你有一套完整的单元测试，让你对单个模块的正确性有信心，那么你就可以少走很多弯路来发现错误。

假设你正在构建一个文档搜索引擎。你的两个模块可能是 load() 和 extract()，load()用于加载文件，extract()用于将文档分割成单词：

```ts
/**
 * @returns the contents of the file
 */
function load(filename: string): string { ... }

/**
 * @returns the words in string s, in the order they appear,
 *         where a word is a contiguous sequence of
 *         non-whitespace and non-punctuation characters
 */
function extract(s: string): Array<string> { ... }
```

这些函数可能被另一个模块index()用来创建搜索引擎的索引：

```ts

/**
 * @returns an index mapping a word to the set of filenames
 *         containing that word, for all files in the input set
 */
function index(filenames: Set<string>): Map<string, Set<string>> {
    ...
    for (const file of files) {
        const doc = load(file);
        const words = extract(doc);
        ...
    }
    ...
}

```

在我们的测试套件中，我们需要

* 对各种文件进行load的单元测试
* 在各种字符串上测试extract的单元测试
* 在不同的文件集上测试index的单元测试

程序员有时会犯的一个错误是，在编写extract的测试用例时，测试用例的正确性依赖于load。例如，一个测试用例可能使用load来加载一个文件，然后将其结果作为输入传递给extract。但这不是extract的单元测试。如果测试用例失败了，我们就不知道失败是由于加载的错误还是extract的错误。

最好是单独考虑和测试extract。使用涉及真实文件内容的测试分区可能是合理的，因为这就是程序中实际使用extract的方式。但是不要在测试用例中实际调用load，因为load可能会有错误！相反，将文件内容存储为字面字符串，并直接传递给extract。这样你就在写一个隔离的单元测试，如果失败了，你可以更确信错误是在实际测试的模块中。

需要注意的是，index的单元测试不容易用这种方法隔离。当测试用例调用 index 时，它不仅测试 index 内部代码的正确性，还测试 index 调用的所有函数的正确性。如果测试失败，错误可能出现在这些函数中的任何一个。这就是为什么我们要对extract和load进行单独测试，以增加我们对这些模块的信心，并将问题定位到将它们连接在一起的索引代码上。

如果我们为index调用的模块编写stub versioon，隔离像index这样的高层模块是可能的。例如，load的stub version根本不会访问文件系统，而是返回mock file的内容，无论传递给它的是什么文件。类的stub 通常被称为mock object。stubs是构建大型系统时的一项重要技术。

```ts
// Mock Object
class MockFile {
  getContent() {
    return 'Mock file content';
  }
}

// Stub
function loadFileContent(file: MockFile): string {
  // Simulating file loading
  return 'Stubbed file content';
}


// Test example
const file = new MockFile();
const content = loadFileContent(file);
console.log(content); // Output: 'Stubbed file content'

```

stub version:
```ts
function index(filenames: Set<string>): Map<string, Set<string>> {
    ...
    for (const file of files) {
        const doc = loadFileContent(file);
        const words = extract(doc);
        ...
    }
    ...
}
```


## Automated regression testing

一旦您实现了测试自动化，在修改代码时重新运行测试是非常重要的。软件工程师从痛苦的经验中知道，对大型或复杂程序的任何修改都是危险的。无论您是要修复另一个错误、添加一个新功能，还是要优化代码使其更快，一个能够保留正确行为基线的自动化测试套件--即使只有几个测试--都会拯救您的生命。当你修改代码时，经常运行测试可以防止你的程序倒退--当你修复新的bug或添加新的功能时，引入其他bug。在每次更改后运行所有测试被称为回归测试。

每当您发现并修复一个错误时，将引起错误的输入作为一个测试用例添加到您的自动化测试套件中。这种测试用例称为回归测试。这有助于用好的测试用例填充您的测试套件。请记住，如果一个测试引出了一个错误，那么它就是好的测试--每个回归测试都在您的代码的一个版本中引出了一个错误！保存回归测试还可以防止重新引入错误的还原。这个错误可能很容易犯，因为它已经发生过一次了。

这个想法也导致了测试优先的调试。当一个bug出现时，立即为它写一个测试用例来引起它，并立即将它添加到你的测试套件中，用一个名称来标识它来自一个bug报告而不是分区，例如it("cover bug #1079")。一旦你发现并修复了这个bug，你的所有测试用例都将通过，你就完成了调试，并有了针对这个bug的回归测试。

在实践中，自动化测试和回归测试这两个概念几乎总是结合使用。只有当测试可以经常自动运行时，回归测试才是实用的。相反，如果您的项目已经有了自动化测试，那么您可以使用它来防止回归。因此，自动化回归测试是现代软件工程的最佳实践。

以下哪一个是重新运行所有测试的好时机？

- [x] 在执行 git add/commit/push 之前
- [x] 重写函数以使其更快之后
- [x] 使用代码覆盖率工具时
- [x] 当你认为你修复了一个错误之后

将代码推送到 git 会将其发送给团队的其他成员，因此首先重新运行测试以确保您没有推送损坏的代码。

重写函数可能会引入错误，因此请重新运行测试以查找它们。

重新运行测试是使用代码覆盖工具的重要组成部分，因为您希望查看测试未到达的代码行。

修复错误就是对程序的更改，每次更改后都应该重新运行测试。



## Iterative test-first programming
让我们重温一下本文开头介绍的 "测试优先 "编程思想，并对其进行细化。有效的软件工程并不遵循线性过程。实践 "测试优先 "的迭代编程，在这个过程中，您准备好返回并修改之前步骤中的工作：

1. 为函数编写规范。
2. 编写测试来验证规范。当您发现问题时，迭代规范和测试。
3. 编写实现。当您发现问题时，反复检查规范、测试和实现。
   
每一步都有助于验证前面的步骤。编写测试是理解规范的好方法。规范可能是不正确的、不完整的、模棱两可的，或者缺失了corner case。尝试编写测试可以及早发现这些问题，避免浪费时间去实现有缺陷的规范。同样，编写实现可以帮助你发现遗漏的或不正确的测试，或者促使你重新审视和修改规范。

由于可能有必要对之前的步骤进行迭代，因此在进入下一步之前花费大量时间使一个步骤变得完美是没有意义的。

制定迭代计划：

* 对于一个大的规范，开始时只写规范的一部分，接着测试和实现这一部分，然后用一个更完整的规范进行迭代。

* 对于复杂的测试套件，首先选择几个重要的部分，并为其创建一个小型测试套件。进行简单的实现以通过这些测试，然后使用更多的分区对测试套件进行迭代。

* 对于棘手的实现，首先编写一个简单的暴力实现，测试您的规范并验证您的测试套件。然后，在确信您的规范是好的，测试是正确的情况下，继续编写更难的实现。

迭代是每个现代软件工程过程（如Agile和Scrum）的一个特征，其有效性得到了良好的经验支持。迭代需要一种不同于学生解决家庭作业和考试问题的心态。迭代意味着尽快找到一个粗略的解决方案，然后不断完善和改进，以便在必要时有时间放弃和返工，而不是试图从头到尾完美地解决一个问题。当问题比较棘手、解决空间未知时，迭代可以最大限度地利用时间。


### reding exercise

假设您正在编写一个二分搜索函数，并且您不仅需要提供一个有效的实现，还需要为客户端提供清晰的规范以及有用的测试套件。

尽管二分搜索是一种易于理解的简单算法，但众所周知，正确实现它非常困难，因此您正在寻找可以获得的所有帮助以确保正确实现。

在实现二分搜索算法之前，以下哪些步骤将有助于验证您的规范？

- [x] 黑盒测试
- [ ] 玻璃盒测试
- [ ] 运行测试
- [x] 编写一个简单的线性搜索算法

黑盒测试通过探索其行为和编写客户端（测试用例）来验证规范。但glass test测试更侧重于实现。

运行测试不太可能发现规范中的错误。

编写一个简单的线性搜索算法将从实现者的角度来执行规范——实现者是否拥有完成这项工作所需的所有信息？

--- 

在实现二分搜索算法之前，以下哪些步骤将有助于验证您的测试套件？

- [x] 编写一个简单的线性搜索算法
- [x] 在简单的实现上运行代码覆盖率工具
- [x] 通过运行 TypeScript 编译器进行静态类型检查


所有这三种技术都可以发现有缺陷的测试或不完整的测试。

--- 

为什么在开始棘手的实现之前找到并删除规范和测试套件中的错误是件好事？

- [x] 这样当出现错误时，您可以假设错误可能出现在实现代码中，而不是规范或测试中
- [x] 因为修复规范中的错误可能会迫使实施发生重大变化
- [ ] 因为修复测试套件中的错误可能会迫使实施发生重大变化


首先使用简单的线性搜索算法调试规范和测试，可以让您更加确信规范和测试是正确的。然后，如果您在编写棘手的实现时遇到错误，您可以对规范和测试进行无罪推论，并将调试工作集中在刚刚编写的实现代码上。

仅更改测试套件（而不更改规范）不会强制更改实现。

## summary
在这篇阅读中，我们看到了这些想法：

* 测试优先的编程。在编写代码之前先编写测试。
* 使用分区和边界值进行系统测试，以设计正确、彻底且小型的测试套件。
* 在测试套件中记录glass test 和 statement覆盖
* 尽可能隔离地对每个模块进行单元测试。
* 自动回归测试可防止错误再次出现。
* 迭代开发。计划重做一些工作。

The topics of today’s reading connect to our three key properties of good software as follows:

* Safe from bugs. Testing is about finding bugs in your code, and test-first programming is about finding them as early as possible, right after you introduce them.

* Easy to understand. Systematic testing with a documented testing strategy makes it easier to understand how test cases were chosen and how thorough a test suite is.

* Ready for change. Correct test suites only depend on behavior in the spec, which allows the implementation to change within the confines of the spec. We also talked about automated regression testing, which helps keep bugs from coming back when changes are made to code.